# Create test set

In the file `explore_data.ipynb`, I found that a token cutoff of 3000 had over 95% of all tokens and articles. I also found that a cutoff of 30% symbol tokens captures about 90% of tokens and articles. In order to have a test set that is not to onerous to produce or too poor quality to recover I am going to combine both of these metrics

Need to do
- Load all articles, keep only the number of tokens, the page and the number of symbols
- sum to page level
- calc fract symbols
- review distribution
- minimum token count, maximum symbol fract
- subset pages
- sample

In [1]:
import pandas as pd
import tiktoken
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import time  # Make sure to import the time module
import shutil
import PyPDF2

from helper_functions import identify_file, find_pdf_path, extract_pages_from_pdf, process_pdfs


from dotenv import load_dotenv, find_dotenv
load_dotenv()

directory = './data/ncse_text_chunks'

dev_transcripts = 'data/dev_data_transcript'

dev_gpt4_results = 'data/dev_data_gpt-4-turbo-preview'
dev_gpt3_results = 'data/dev_data_gpt-3.5-turbo'

image_path = os.getenv("image_path")



Get the conversion from the publication ID to the folder path of the archived images from the NCSE figshare

In [2]:
folder2id_df = pd.DataFrame([{'folder':'English_Womans_Journal_issue_PDF_files', 'publication_id':24},
 {'folder':'Leader_issue_PDF_files/Leader_issue_PDF_files', 'publication_id':20},
 {'folder':'Monthly_Repository_issue_PDF_files', 'publication_id':22},
 {'folder':'Northern_Star_issue_PDF_files', 'publication_id':27},
 {'folder':'Publishers_Circular_issue_PDF_files', 'publication_id':26},
 {'folder':'Tomahawk_issue_PDF_files/Tomahawk_issue_PDF_files', 'publication_id':19}])

In [3]:
page_list = pd.read_parquet('data/periodicals_page.parquet')

periodicals_issue = pd.read_parquet('data/periodicals_issue.parquet')

periodicals_publication = pd.read_parquet('data/periodicals_publication.parquet')

In [4]:
page_info = page_list.rename(columns = {'number':'page_number'}).merge(
    periodicals_issue[['id', 'publication_id', 'issue_date']].rename(columns = {'id':'issue_id'}), on = 'issue_id'
).merge(
   periodicals_publication[['id', 'slug', 'title']].rename(columns = {'id':'publication_id'}), on='publication_id' 
)


In [5]:
np.random.seed(1842)
pages_for_transcription_1 = page_info.groupby('title').sample(10).merge(folder2id_df, on = 'publication_id')

pages_for_transcription_1['pdf_path'] =pages_for_transcription_1.apply(lambda row: find_pdf_path(image_path, row['folder'],  row['issue_date'].strftime('%Y-%m-%d')), axis=1)
pages_for_transcription_1['file_name'] = pages_for_transcription_1['pdf_path'].apply(os.path.basename)


process_pdfs(pages_for_transcription_1, output_folder="data/pdfs_for_transcription")